In [64]:
'''
1.将三个机构的销售订单进行concat后按产品品种汇总
2.将工厂的销售订单按产品品种汇总，汇总前过滤到销售单位是销售公司的数据
3.将上面两个汇总进行对比，找出有差异的品种，计算出差异本数、差异件数、差异金额
销售订单都是完整引出（从销售明细中完整引出）
'''
import os
import datetime
import easygui
import numpy as np
import pandas as pd
import openpyxl
from openpyxl.styles import Font, Border, Side, Fill, Alignment
import formatPainter
import re

#工厂的列标题
lst_columns = [
 '单据编号',
  '单据日期',
  '存货编码',
 '规格型号',
 '数量',
 '数量2'
  ]

#机构的列标题有点不一样
lst_columns_jigou = [
   '存货编码',
 '规格型号',
 '数量(主单位)',
 '数量(辅单位)'
]
def chuliXiaoshouDingdan(fname,lst_columns):
    df = pd.read_excel(fname,header = 7)
    df = df.iloc[:,1:]
    df = df[lst_columns]
    df = df.set_index('单据日期')
    df = df.loc[:'合计：',:]
    df = df.iloc[:-1]
    df = df.reset_index()
    df = df.rename(columns = {'数量':'本数','数量2':'件数'})
    return df

def XiaoshoubuDingdanHuizhaong(fname,lst_columns_jigou):
    df = pd.read_excel(fname,header = 7)
    df = df.iloc[:,1:]
    df = df[lst_columns_jigou]
    df = df.set_index('规格型号')
    df = df.loc[:'合计：',:]
    df = df.iloc[:-1]
    df = df.reset_index()
    df= df.rename(columns = {'数量(主单位)':'本数','数量(辅单位)':'件数'})
    df = df.set_index( ['存货编码','规格型号'])
    return df







In [65]:
#处理销售公司的销售订单统计表（含三个机构）
fname_gongchang = r"F:\a00nutstore\006\zw\产成品日报表\2024年\2024年9月\9月6日-8日\销售订单明细表9.6-9.8未审核.xlsx"
df_gongchang = chuliXiaoshouDingdan(fname_gongchang,lst_columns)
df_gongchang

,单据日期,单据编号,存货编码,规格型号,本数,件数
0,2024-09-07,SO-2024-2017,N0787（电商）a,B20电商缝线本,2000.0,10.0
1,2024-09-07,SO-2024-2016,ATHA460N,C5锐意美术系列,420.0,3.0
2,2024-09-07,SO-2024-2016,RFBJ-A460N,D1新锐缝线本,140.0,1.0
3,2024-09-07,SO-2024-2015,N0787（电商）a,B20电商缝线本,4000.0,20.0
4,2024-09-07,SO-2024-2014,N0813,B1软抄,1440.0,4.0
...,...,...,...,...,...,...
366,2024-09-06,SO-2024-1936,N0780,B12莱特牛皮缝线本,200.0,1.0
367,2024-09-06,SO-2024-1936,N0795,B12莱特牛皮缝线本,640.0,2.0
368,2024-09-06,SO-2024-1936,NFC1660NH（方格）-1,C4锐意专利缝线本-黄内芯,320.0,2.0
369,2024-09-06,SO-2024-1936,NFY1660NH（双色）-1,C4锐意专利缝线本-黄内芯,640.0,4.0


In [66]:
#处理三个机构的销售订单
fname_xiaoshou  = r"F:\a00nutstore\006\zw\产成品日报表\2024年\2024年9月\9月6日-8日\9.6-9.8销售订单统计表-销售部.xlsx"
df_xiaoshou = XiaoshoubuDingdanHuizhaong(fname_xiaoshou,lst_columns_jigou)
df_xiaoshou

,,本数,件数
存货编码,规格型号,,
XJ415,B8小学生作业本,840.0,3
XJ409,B8小学生作业本,280.0,1
XJ407,B8小学生作业本,840.0,3
XJ404S,B8小学生作业本,1960.0,7
XJ403S,B8小学生作业本,840.0,3
...,...,...,...
2001,A4凭证,1080.0,4
16K备课本-50Pa,B16电商备课本,1800.0,10
1117,A2账芯,120.0,2


In [67]:
df_gongchangchangPivot = pd.pivot_table(df_gongchang,index = ['存货编码','规格型号'],values = ['本数','件数'],fill_value =0,aggfunc =np.sum)
df_gongchangchangPivot

C:\Users\redda\AppData\Local\Temp\ipykernel_19580\2469539234.py:1: FutureWarning: The provided callable <function sum at 0x0000020A32037380> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  df_gongchangchangPivot = pd.pivot_table(df_gongchang,index = ['存货编码','规格型号'],values = ['本数','件数'],fill_value =0,aggfunc =np.sum)


,,件数,本数
存货编码,规格型号,,
1106,A2账芯,1.0,60.0
1110,A2账芯,2.0,120.0
1117,A2账芯,2.0,120.0
16K备课本-50Pa,B16电商备课本,10.0,1800.0
2001,A4凭证,4.0,1080.0
...,...,...,...
XJ403S,B8小学生作业本,3.0,840.0
XJ404S,B8小学生作业本,7.0,1960.0
XJ407,B8小学生作业本,3.0,840.0


In [68]:
df_diff = pd.merge(df_gongchangchangPivot,df_xiaoshou,how = "outer",left_index = True,right_index = True)
df_diff = df_diff.rename(columns = {'件数_x':'工厂件数','本数_x':'工厂本数'	,'本数_y':'销售本数','件数_y':'销售件数'})
df_diff = df_diff[['工厂件数','工厂本数','销售件数','销售本数']]
df_diff = df_diff.fillna(0)
df_diff['件数差异'] = df_diff['工厂件数'] - df_diff['销售件数']
df_diff['本数差异'] = df_diff['工厂本数'] - df_diff['销售本数']

C:\Users\redda\AppData\Local\Temp\ipykernel_19580\489602838.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_diff = df_diff.fillna(0)


In [70]:
df_diff

,,工厂件数,工厂本数,销售件数,销售本数,件数差异,本数差异
存货编码,规格型号,,,,,,
1106,A2账芯,1.0,60.0,1.0,60.0,0.0,0.0
1110,A2账芯,2.0,120.0,2.0,120.0,0.0,0.0
1117,A2账芯,2.0,120.0,2.0,120.0,0.0,0.0
16K备课本-50Pa,B16电商备课本,10.0,1800.0,10.0,1800.0,0.0,0.0
2001,A4凭证,4.0,1080.0,4.0,1080.0,0.0,0.0
...,...,...,...,...,...,...,...
XJ403S,B8小学生作业本,3.0,840.0,3.0,840.0,0.0,0.0
XJ404S,B8小学生作业本,7.0,1960.0,7.0,1960.0,0.0,0.0
XJ407,B8小学生作业本,3.0,840.0,3.0,840.0,0.0,0.0


In [63]:
df_diff.to_excel('工厂与销售部订单差异2024-9-6.xlsx')